<a href="https://www.kaggle.com/code/srbharathee/sg-helper-bot?scriptVersionId=235073133" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# SG Helper Bot: Multilingual, Search-Grounded Singapore Assistant

---

## Introduction

This is built as a capstone project for Gen AI Intensive Course Capstone 2025Q1 (https://www.kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1) . The concepts used are from the learning resorces shared during -day Gen AI Intensive Course with Google (https://rsvp.withgoogle.com/events/google-generative-ai-intensive_2025q1)  
  
This notebook demonstrates a **Singapore Helper Bot**—a multilingual AI assistant powered by Google Gemini.  
It can answer queries in English, Chinese, Malay, and Tamil, and provides up-to-date, trustworthy information by grounding its answers in real-time Google Search results.  
The bot is designed for Singaporeans and visitors seeking local, practical, and official information, including transport, food, healthcare, and government services.

**Key Features:**
- Multilingual Q&A (all official SG languages)
- Real-time Google Search grounding with citations
- Localized answers for Singapore context
- Ready for extension with Singapore government open data

---

## Approach

- **Foundation Model:** Uses Gemini (Flash or Pro experimental if available) for LLM-powered Q&A and translation.
- **Grounding:** Gemini’s built-in Google Search grounding ensures answers are up-to-date and cite sources.
- **Multilingual:** Accepts and responds in any of Singapore’s official languages, using prompt engineering and optional language detection.


---

## 1. Environment and Package Setup



In [ ]:
!pip show google-genai



**We use `google-generativeai` for responses without grounding and google-genai for results with google search grounding . This is to try the different modules available .An exhaustive comparison of the modules has not been done yet**

In [ ]:
#!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
#!pip install -U -q "google-genai==1.7.0"
!pip install -q google-generativeai

## 2. Obtain Google API key 

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, HTML, display

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
import google.generativeai as genai
genai.configure()  # This will pick up the API key from the environment

#or do 
'''genai.configure(
    api_key=GOOGLE_API_KEY,
    
)'''


**This cell prints all available Gemini models.  
You can choose the latest or most suitable one for your use case.**


In [ ]:
#if we wanted to list the models to make a selection . commented out for now
#for m in genai.list_models():
#    print(m)

## 3. Define Multilingual Helper Function



In [ ]:
from google.generativeai import GenerativeModel
model = GenerativeModel('gemini-2.5-pro-exp-03-25')




def multilingual_helper(user_input, target_language=None):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language: {user_input}"
    response = model.generate_content(prompt)
    return response.text

# Usage example
print(multilingual_helper("请问附近哪里有素食餐厅？"))  # Chinese input
print(multilingual_helper("Where can I find vegan food near Bugis?", target_language="Malay"))


## Few Shot examples 

In [ ]:
few_shot_examples = """
Example 1:
Q: Where can I find vegan food near Bugis?
A: You can find vegan food at Fortune Centre, which has several vegan and vegetarian restaurants. Other options near Bugis include Loving Hut and Herbivore.

Example 2:
Q: 如何从裕廊东去牛车水？
A: 您可以搭乘地铁东西线（绿色线）从裕廊东站直接到牛车水站，中途在欧南园站换乘地铁东北线（紫色线）。

Example 3:
Q: Di mana saya boleh dapatkan makanan India halal di Yishun?
A: Anda boleh cuba restoran seperti Al-Azhar Eating Restaurant di Yishun atau kedai-kedai mamak di sekitar kawasan tersebut.
"""


In [ ]:
def multilingual_helper_few_shot(user_input):
    prompt = f"""{few_shot_examples}

Now, answer the following question in the same style as above:
Q: {user_input}
A:"""
    response = model.generate_content(prompt)
    return response.text


In [ ]:
print(multilingual_helper_few_shot("What are the CPF withdrawal rules at age 55?"))
print(multilingual_helper_few_shot("请问从三巴旺怎么去樟宜机场？"))


## 4. Install Language detection and add language detection capability

**While we could use Google genai capability to detect the language, detecting the language offline /locally is much more cost effective and efficient**

In [ ]:
!pip install -q langdetect

In [ ]:
from langdetect import detect

def detect_and_respond(user_input):
    lang = detect(user_input)
    return multilingual_helper(user_input, target_language=lang)


## Interactive User query input loop . commented out for submission 

In [ ]:
#commented out to keep this reference run during submission
'''while True:
    user_query = input("Ask me anything (type 'quit' to exit): ")
    if user_query.lower() == 'quit':
        break
    print("Bot:", detect_and_respond(user_query))'''

## 5. Sample queries with multilingual response in all of Singapore official languages 

In [ ]:
user_query="Can you recommend laptop service centers around Kembangan to fix a Dell Laptop screen. Provide answers in Malay and English "
from IPython.display import Markdown, HTML, display
response_text= detect_and_respond(user_query)
display(Markdown(f"**Sg Helper Bot:** {response_text}"))


In [ ]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond(user_query)
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

## Structured output in JSON 

**Let us say that we want to send the response back as a JSON , for instance as a response to an API call, use this structured output feature**

In [ ]:
from google.genai import types
from IPython.display import JSON

config_json= types.GenerateContentConfig(response_format='json')

def multilingual_helper_structured(user_input, target_language=None):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language: {user_input}"
    response = model.generate_content(prompt)
    return response.text

def detect_and_respond_strcutured(user_input):
    lang = detect(user_input)
    return multilingual_helper(user_input, target_language=lang)


In [ ]:
user_input="Get me a trend of the housing prices in Singapore. Provide answers in all official languages "
response_text=detect_and_respond_strcutured(user_input)
JSON(response_text)

## 6. Enable Grounding with Google Search 

**We use google-genai v1.7.0 for this section** 

In [ ]:
#google.generativeai' has no attribute 'Tool' Need to explore more for grounding
#use google-genai

# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

In [ ]:
#enable grounding 
from google import genai
from google.genai import types

from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, HTML, display

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)









## 7. Retry mechanism - to use when client retries if it hits quota limits 

In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)


## 8. multilingual helper with grounding enabled 

*This takes a model_to_use param so that different models can be called* 

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)


def multilingual_helper_with_grounding(user_input, target_language=None,model_to_use='gemini-2.0-flash'):
    if target_language:
        prompt = f"Translate the following and answer in {target_language}: {user_input}"
    else:
        prompt = f"Detect the language and answer in the same language: {user_input}"
    #response = model_with_grounding.generate_content(prompt,grounding_source=genai.GroundingSource.GOOGLE_SEARCH)
    response = client.models.generate_content(
        model=model_to_use,
        contents=prompt,
        config=config_with_search,
    )
    return response.text

def detect_and_respond_with_grounding(user_input,model_to_use='gemini-2.0-flash'):
    lang = detect(user_input)
    return multilingual_helper_with_grounding(user_input, target_language=lang,model_to_use=model_to_use)



## 9. Query using gemini-2.0-flash (default model in helper fn) and get Multilingiual response in all Sg official languages 

In [ ]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond_with_grounding(user_query) #use gemini_2.0-flash
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

## 10. get multilingual response for the same query using gemini-2.5-pro-exp-03-25 

In [ ]:
user_query="Get me a trend of the housing prices in Singapore. Provide answers in all official languages"
response_text= detect_and_respond_with_grounding(user_query,'gemini-2.5-pro-exp-03-25') #use gemini-2.5-pro-exp-03-25
display(Markdown(f"**Sg Helper Bot:** {response_text}"))

## Comparing responses from the two models using grounding with Google Search . 
## While the response is better formatted with gemini-2.5-pro-exp-03-25 using Google search grounding , its reference time frame is the first quarter of 2024!!
## Gemini-Flash-2.0 seems to have a more recent reference point , the answers were more concise. 
## Need to explore this aspect further 

****Extending the Bot:
You can add Data.gov.sg or other APIs as tools for even richer, hyperlocal answers.
Evaluation:
Try queries like “Where can I get halal food near Bugis?”, “List clinics open near Kembangan”****